In [1]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import os

In [ ]:
# requests
file_path = r"C:\Users\ElifYilmaz\Downloads\project 2.0.csv"

df = pd.read_csv(file_path)
df.columns = df.columns.str.lower()

#older contracts
path = r"C:\Users\ElifYilmaz\Downloads\project 2.0_contracts.csv"
df2 = pd.read_csv(path)
df2.columns = df.columns.str.lower()

#concat
combined_df = pd.concat([df, df2], ignore_index=True)
df = combined_df.copy()

# qualitative 

file2 = r"C:\Users\ElifYilmaz\Downloads\info client.csv"

df2 = pd.read_csv(file2)
df2.columns = df2.columns.str.lower()

In [3]:
# drop duplicates in df2,they were found in notebook 01
# Step 1: Count requestids that have duplicates
duplicate_requestids = df2[df2['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Number of requestids with duplicates: {len(duplicate_requestids)}")
print(f"Total duplicate rows: {df2['requestid'].duplicated().sum()}")

# Step 2: Drop absolute duplicates (excluding leadid column)
comparison_cols = [col for col in df2.columns if col != 'leadid']
df2_before = len(df2)
df2 = df2.drop_duplicates(subset=comparison_cols, keep='first')
df2_after = len(df2)

print(f"\nDropped {df2_before - df2_after} absolute duplicate rows")

# Step 3: From the remaining duplicate requestids, drop rows where netcontractsigned = 0
remaining_duplicate_requestids = df2[df2['requestid'].duplicated(keep=False)]['requestid'].unique()

if len(remaining_duplicate_requestids) > 0:
    print(f"Remaining duplicate requestids before netcontractsigned cleanup: {len(remaining_duplicate_requestids)}")
    
    # Create condition: either NOT a duplicate requestid, OR netcontractsigned != 0
    condition = (~df2['requestid'].isin(remaining_duplicate_requestids)) | (df2['netcontractsigned'] != 0)
    
    df2_before_net_drop = len(df2)
    df2 = df2[condition]
    df2_after_net_drop = len(df2)
    
    print(f"Dropped {df2_before_net_drop - df2_after_net_drop} rows where netcontractsigned = 0 from duplicate requestids")

# Step 4: Final duplicate check
print("\n" + "="*50)
print("FINAL DUPLICATE CHECK")
print("="*50)

final_duplicate_requestids = df2[df2['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Number of requestids with duplicates: {len(final_duplicate_requestids)}")
print(f"Total duplicate rows: {df2['requestid'].duplicated().sum()}")

if len(final_duplicate_requestids) > 0:
    # Check what columns still differ
    all_differing_columns = set()
    for requestid in final_duplicate_requestids:
        group = df2[df2['requestid'] == requestid]
        for col in df2.columns:
            if col != 'requestid' and group[col].nunique() > 1:
                all_differing_columns.add(col)
    
    print(f"Differing columns: {sorted(all_differing_columns)}")
else:
    print(f"✅ No remaining duplicates!")

Number of requestids with duplicates: 35
Total duplicate rows: 35

Dropped 8 absolute duplicate rows
Remaining duplicate requestids before netcontractsigned cleanup: 27
Dropped 27 rows where netcontractsigned = 0 from duplicate requestids

FINAL DUPLICATE CHECK
Number of requestids with duplicates: 0
Total duplicate rows: 0
✅ No remaining duplicates!


#### Create Derived Features

In [4]:
# fixing appointment types

type_mapping = {
    'FU': 'FU',
    'FUVD': 'FU', 
    'REMINDER': 'FU',
    'VDFIELD': 'SC',
    'VD': 'SC'
}

# Create grouped_type column
df['grouped_type'] = df['type'].map(type_mapping)

In [5]:
# Create condition for net events (not cancelled and past)
df['is_net_event'] = df['deletedat'].isna() & df['is_passed']

In [6]:
# Gross counts (all events)
gross_counts = df.groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
gross_counts.columns = [f'gross_{col}' for col in gross_counts.columns]

# Net counts (only events that actually happened)
net_counts = df[df['is_net_event']].groupby(['requestid', 'grouped_type']).size().unstack(fill_value=0)
net_counts.columns = [f'net_{col}' for col in net_counts.columns]

In [7]:
# Combine gross and net counts
counts_df = gross_counts.join(net_counts, how='outer').fillna(0)

In [8]:
counts_df.index.duplicated().any()

np.False_

In [9]:
# turn self ipa event to bbinary 
df2['selfipa_done'] = df2['selfipaimportedat'].notnull().astype(int)
df2.drop('selfipaimportedat', axis=1, inplace=True) 

In [10]:
# Step 1: Convert createdat to datetime
df['createdat'] = pd.to_datetime(df['createdat'])

# Step 2: Get first gross SC and first net FU timestamps
first_gross_sc = df[df['grouped_type'] == 'SC'].groupby('requestid')['createdat'].min()
first_net_fu = df[(df['grouped_type'] == 'FU') & df['is_net_event']].groupby('requestid')['createdat'].min()

# Step 3: Calculate time difference in hours
counts_df['time_first_sc_to_first_net_fu'] = (first_net_fu - first_gross_sc).dt.total_seconds() / 3600

In [11]:
# drop older requests that havent had a sc since march
counts_df = counts_df[counts_df['gross_SC'] != 0]
counts_df['gross_SC'].value_counts() 

gross_SC
1     11241
2      2185
3       532
4       148
5        34
6         8
7         5
10        2
23        1
Name: count, dtype: int64

In [12]:
# handle missing value for customers who never made to fu
counts_df['time_first_sc_to_first_net_fu'] = counts_df['time_first_sc_to_first_net_fu'].fillna(-1)

# Check final dataset
print("Final dataset info:")
print(f"Shape: {counts_df.shape}")
print(f"No missing values: {counts_df.isnull().sum().sum() == 0}")

Final dataset info:
Shape: (14156, 5)
No missing values: True


In [13]:
# problematic entries with negative time difference: 
# Separate real negative times from NaN fill values
real_negative = counts_df[
    (counts_df['time_first_sc_to_first_net_fu'] < 0) & 
    (counts_df['time_first_sc_to_first_net_fu'] != -1)
]

actual_valid_times = counts_df[counts_df['time_first_sc_to_first_net_fu'] >= 0]

print(f"Actual negative time differences (not NaN fills): {len(real_negative)}")
print(f"Positive time differences: {len(actual_valid_times)}")
print(f"NaN fills (-1): {(counts_df['time_first_sc_to_first_net_fu'] == -1).sum()}")

if len(real_negative) > 0:
    print(f"\nReal negative values range: {real_negative['time_first_sc_to_first_net_fu'].min():.2f} to {real_negative['time_first_sc_to_first_net_fu'].max():.2f} hours")


Actual negative time differences (not NaN fills): 7
Positive time differences: 5964
NaN fills (-1): 8185

Real negative values range: -905.06 to -23.08 hours


In [14]:
#handle negative time difference
# Remove outliers entirely (set to NaN, then fill with -1)
counts_df.loc[counts_df['time_first_sc_to_first_net_fu'] < -1, 'time_first_sc_to_first_net_fu'] = -1

In [15]:
# Verify the cleanup
print("After removing outliers:")
print(f"Negative values (excluding -1 fills): {((counts_df['time_first_sc_to_first_net_fu'] < 0) & (counts_df['time_first_sc_to_first_net_fu'] != -1)).sum()}")
print(f"Valid positive time differences: {(counts_df['time_first_sc_to_first_net_fu'] > 0).sum()}")
print(f"NaN fills (-1): {(counts_df['time_first_sc_to_first_net_fu'] == -1).sum()}")

# Final dataset summary
print(f"\nFinal dataset shape: {counts_df.shape}")
print("Ready to join with target variable!")

After removing outliers:
Negative values (excluding -1 fills): 0
Valid positive time differences: 5964
NaN fills (-1): 8192

Final dataset shape: (14156, 5)
Ready to join with target variable!


In [16]:
# at this point requestid is the index. make it explicit


counts_df = counts_df.reset_index()
counts_df.head()


,requestid,gross_FU,gross_SC,net_FU,net_SC,time_first_sc_to_first_net_fu
0,3279,0,1,0.0,1.0,-1.000000
1,3729,8,1,8.0,1.0,73.835556
2,3852,2,1,2.0,1.0,42.193889
3,4359,0,1,0.0,0.0,-1.000000
4,4689,4,1,1.0,1.0,837.203889


In [17]:
# requestid duplicates? --> no 
# Show which requestid values appear more than once
duplicate_requestids = counts_df[counts_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [18]:
# also df2 has no duplicates
duplicate_requestids = df2[df2['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [19]:


final_df = counts_df.merge(df2, on='requestid', how='left')


# Fill non-matching requestids with 0 for the target
final_df['netcontractsigned'] = final_df['netcontractsigned'].fillna(0)

# Check the result
print(f"Final dataset shape: {final_df.shape}")
print(f"Target variable distribution:")
print(final_df['netcontractsigned'].value_counts())
print(f"\nColumns: {list(final_df.columns)}")

Final dataset shape: (14156, 14)
Target variable distribution:
netcontractsigned
0.0    13812
1.0      344
Name: count, dtype: int64

Columns: ['requestid', 'gross_FU', 'gross_SC', 'net_FU', 'net_SC', 'time_first_sc_to_first_net_fu', 'zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'heatingbill', 'mktgparamscore', 'netcontractsigned', 'selfipa_done']


In [20]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


#### Encoding and Null handling

In [21]:
# Majority of bills have -1, 1 and 0

# set wrong bill values to nan
# Replace specific values with NaN
final_df['electricitybill'] = final_df['electricitybill'].replace([0, 1, -1], pd.NA)
final_df['heatingbill'] = final_df['heatingbill'].replace([0, 1, -1], pd.NA)

Optional for bill handling 
def clean_billing_column(series, column_name):
    """Clean billing column: set negative values and extreme outliers to replacement value"""
    
    # Convert to numeric, coerce errors to NaN
    series_clean = pd.to_numeric(series, errors='coerce')
    
    # Define reasonable bounds (adjust these based on your domain knowledge)
    min_reasonable = 0  # Bills shouldn't be negative
    max_reasonable = 2000  # Adjust based on your market (e.g., €1000/month seems high)
    
    # Count issues for reporting
    negative_count = (series_clean < min_reasonable).sum()
    outlier_count = (series_clean > max_reasonable).sum()
    
    print(f"{column_name}:")
    print(f"  - Negative values: {negative_count}")
    print(f"  - Values > {max_reasonable}: {outlier_count}")
    print(f"  - Original NaN: {series.isna().sum()}")
    
    # Replace problematic values
    series_clean.loc[series_clean < min_reasonable] = np.nan  # or -1
    series_clean.loc[series_clean > max_reasonable] = np.nan  # or -1
    
    return series_clean

# Apply cleaning
final_df['heatingbill'] = clean_billing_column(final_df['heatingbill'], 'heatingbill')
final_df['electricitybill'] = clean_billing_column(final_df['electricitybill'], 'electricitybill')

In [22]:
# before encoding, mark the missingness inside the row;
missing_cols = ['zipregion', 'evaluationtime', 'desiredinstallationend', 'electricitybill', 'heatingbill', 'mktgparamscore']

for col in missing_cols:
    final_df[f'{col}_missing'] = final_df[col].isnull().astype(int)

# Check the predictive power of missing indicators
print("Missing indicator correlations with target:")
missing_indicators = [f'{col}_missing' for col in missing_cols]
missing_corrs = final_df[missing_indicators + ['netcontractsigned']].corr()['netcontractsigned'].drop('netcontractsigned')
print(missing_corrs.sort_values())

Missing indicator correlations with target:
electricitybill_missing          -0.048352
evaluationtime_missing           -0.013618
desiredinstallationend_missing   -0.013618
mktgparamscore_missing           -0.000055
zipregion_missing                 0.015379
heatingbill_missing               0.027626
Name: netcontractsigned, dtype: float64


#### Encoding

In [23]:
# encoding 
# desiredinstallationend
type_mapping = {
    # Original Italian values
    '3-4mesi': 'three_to_four_months', 
    '5+mesi': 'more_than_5_months',
    '1-2mesi': 'one_to_two_months',
    'Non lo so': 'dont_know',
    'short': np.nan,
    # Already mapped values (keep as-is)
    'dont_know': 'dont_know',
    'three_to_four_months': 'three_to_four_months',
    'one_to_two_months': 'one_to_two_months', 
    'more_than_5_months': 'more_than_5_months',
    # Handle string 'nan'
    'nan': np.nan
}

final_df['desiredinstallationend1'] = final_df['desiredinstallationend'].map(type_mapping)

In [24]:
# evaluationtime
type_mapping = {
    # Original Italian values
    '3-6 mesi': np.nan,
    '<3 mesi': np.nan,
    '>6 mesi': np.nan,
    # Already mapped English values
    'less_than_three_months': np.nan,
    'more_than_six_months': np.nan,
    # Other values that appear in your data
    'understand_need': 'understand_need',  # or map to np.nan if you don't want these
    'understand_purchase': 'understand_purchase',  # or map to np.nan if you don't want these
    'evaluation': 'evaluation',  # or map to np.nan if you don't want these
    'curious': 'curious',  # or map to np.nan if you don't want these
    # Handle string 'nan'
    'nan': np.nan
}

# Create grouped_type column
final_df['evaluationtime1'] = final_df['evaluationtime'].map(type_mapping)


In [25]:
# Replace the old columns directly
final_df['desiredinstallationend'] = final_df['desiredinstallationend1']
final_df['evaluationtime'] = final_df['evaluationtime1']

# Drop the temporary columns
final_df = final_df.drop(['desiredinstallationend1', 'evaluationtime1'], axis=1)

In [26]:
# handle nan before encoding
final_df['desiredinstallationend'] = final_df['desiredinstallationend'].fillna('missing')
final_df['evaluationtime'] = final_df['evaluationtime'].fillna('missing')


In [27]:
# Start with ordinal encoding for time-based features

# Ordinal encoding for desiredinstallationend (time-based order)
time_order = [ 'one_to_two_months', 'three_to_four_months', 'more_than_5_months', 'dont_know', 'missing']
final_df['desiredinstallationend_encoded'] = final_df['desiredinstallationend'].map({val: i for i, val in enumerate(time_order)})

##### Analysis to see how to handle marketing and region

In [28]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14156 entries, 0 to 14155
Data columns (total 21 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       14156 non-null  int64  
 1   gross_FU                        14156 non-null  int64  
 2   gross_SC                        14156 non-null  int64  
 3   net_FU                          14156 non-null  float64
 4   net_SC                          14156 non-null  float64
 5   time_first_sc_to_first_net_fu   14156 non-null  float64
 6   zipregion                       13700 non-null  object 
 7   evaluationtime                  14156 non-null  object 
 8   desiredinstallationend          14156 non-null  object 
 9   electricitybill                 9026 non-null   object 
 10  heatingbill                     3080 non-null   object 
 11  mktgparamscore                  13373 non-null  object 
 12  netcontractsigned               

In [29]:
# For each categorical column, see conversion rates
print("Conversion rates by category:\n")

for col in ['zipregion', 'mktgparamscore']:  # replace with your actual column names
    conversion_by_cat = final_df.groupby(col)['netcontractsigned'].agg(['count', 'sum', 'mean'])
    conversion_by_cat.columns = ['total_samples', 'conversions', 'conversion_rate']
    conversion_by_cat = conversion_by_cat.sort_values('conversion_rate', ascending=False)
    
    print(f"\n{col.upper()}:")
    print(conversion_by_cat)
    print(f"Overall variation: {conversion_by_cat['conversion_rate'].std():.4f}")

Conversion rates by category:


ZIPREGION:
                       total_samples  conversions  conversion_rate
zipregion                                                         
Friuli-Venezia Giulia            776         55.0         0.070876
Piemonte                        1179         48.0         0.040712
Lombardia                       2152         64.0         0.029740
Liguria                          349         10.0         0.028653
Veneto                          1105         28.0         0.025339
Emilia-Romagna                  1212         30.0         0.024752
Marche                           260          6.0         0.023077
Toscana                          919         20.0         0.021763
Valle D'Aosta                     47          1.0         0.021277
Lazio                           1296         25.0         0.019290
Trentino-Alto Adige              106          2.0         0.018868
Basilicata                       113          2.0         0.017699
Umbria             

Marketing gave good variance, would like to maintain that information. Region is not that significant but still good. 
However given the * of unique values, in both columns I will opt for grouping instead of each value having its column. 

Instead of 30+ categorical features, you get ~6-8, keeping the predictive power but losing the noise.

Downside; this grouping should occasionally double checked to see if it still makes sense

##### one hot encoding for marketing and region

In [30]:
# Define performance groups
marketing_groups = {
    'High': ['Referral', 'form_classico', 'Organic', 'Affiliation'],
    'Medium': ['Other', 'Google', 'Youtube', 'Mediago', 'TikTok'], 
    'Low': ['Outbrain', 'Meta', 'Taboola', 'd2d']
}

# Create mapping function
def group_marketing(value):
    for group, channels in marketing_groups.items():
        if value in channels:
            return group
    return 'Low'  # fallback

# Apply grouping
final_df['mktg_grouped'] = final_df['mktgparamscore'].apply(group_marketing)

In [31]:
# Group regions by performance
def group_regions(region):
    if region == 'Friuli-Venezia Giulia':
        return 'High_Performer'
    elif region in ['Piemonte', 'Lombardia', 'Emilia-Romagna', 'Veneto']:
        return 'Large_Good'
    elif region in ['Liguria', 'Toscana', 'Valle D\'Aosta']:
        return 'Medium'
    else:
        return 'Other'

final_df['region_grouped'] = final_df['zipregion'].apply(group_regions)

In [32]:
# One-hot encode the grouped categories
final_df = pd.get_dummies(final_df, columns=['mktg_grouped', 'region_grouped'], prefix=['mktg', 'region'])

In [33]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")


Duplicate requestid values: []


df is the log of all calendar_events so a direct join will make our dataset exponentially duplicated.

I will first get the unique requestid --> leadid cpairs and then merge that

In [34]:
requestid_leadid_pairs = df[['requestid', 'leadid']].drop_duplicates()

In [35]:
requestid_leadid_pairs[requestid_leadid_pairs['requestid'].duplicated(keep=False)]['requestid'].unique()

array([], dtype=int64)

In [36]:
# also add leadid to the final df for merging with behaviour data

# Suppose the column you want from df1 is called "col_from_df1"
final_df = final_df.merge(
    requestid_leadid_pairs[['requestid', 'leadid']],  # keep only requestid + desired column
    on='requestid',                      # join key
    how='left'                           # keep all rows from df2
)


In [37]:
duplicate_requestids = final_df[final_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

Duplicate requestid values: []


In [38]:
# any requestid with no leadid?
final_df[final_df['leadid'].isnull()]['requestid'].unique()

array([], dtype=int64)

In [39]:
# Drop original categorical columns and create final feature matrix
columns_to_drop = [ 'zipregion', 'evaluationtime', 'desiredinstallationend', 'mktgparamscore']
final_df = final_df.drop(columns=columns_to_drop)


print(f"\nNo missing values: {final_df.isnull().sum().sum() == 0}")


No missing values: False


#### Imputing

In [40]:
# Fix electricitybill missing values
final_df['electricitybill'] = final_df['electricitybill'].fillna(final_df['electricitybill'].median())
final_df['heatingbill'] = final_df['heatingbill'].fillna(final_df['heatingbill'].median())

# Fix desiredinstallationend_encoded - NaN means 'Unknown' which should be 0 --> check if needed
#X_final['desiredinstallationend_encoded'] = X_final['desiredinstallationend_encoded'].fillna(0)

# Verify all missing values are gone
print(f"Remaining missing values: {final_df.isnull().sum().sum()}")
print(f"Dataset ready: {final_df.isnull().sum().sum() == 0}")



Remaining missing values: 0
Dataset ready: True


C:\Users\ElifYilmaz\AppData\Local\Temp\ipykernel_20316\1109642794.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['electricitybill'] = final_df['electricitybill'].fillna(final_df['electricitybill'].median())
C:\Users\ElifYilmaz\AppData\Local\Temp\ipykernel_20316\1109642794.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df['heatingbill'] = final_df['heatingbill'].fillna(final_df['heatingbill'].median())


##### Scaling

In [41]:
# scaling when needed
continuous_cols= [
    'time_first_sc_to_first_net_fu',
    'electricitybill', 
    'heatingbill'
]

# Scale only the continuous features
scaler = StandardScaler()
final_df[continuous_cols] = scaler.fit_transform(final_df[continuous_cols])

In [42]:
behaviour = pd.read_csv(r"processed_data/df_model.csv") # manualy saved leadtime info. hafter having gotten modified in older attempts/notebook
behaviour = behaviour.drop(['converted', 'lead_to_sc1_days', 'sc1_schedule_to_appointment_days', 'showed_up_sc1' ], axis=1)

In [43]:
behaviour.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38367 entries, 0 to 38366
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   ID                          38367 non-null  int64  
 1   total_bc_attempts           38367 non-null  int64  
 2   total_bc_outcomes           38367 non-null  int64  
 3   lead_to_first_bc_days       38367 non-null  float64
 4   bc_duration_days            38367 non-null  float64
 5   bc_frequency                38367 non-null  float64
 6   positive_outcomes_count     38367 non-null  int64  
 7   negative_outcomes_count     38367 non-null  int64  
 8   noshow_outcomes_count       38367 non-null  int64  
 9   positive_outcome_ratio      38367 non-null  float64
 10  negative_outcome_ratio      38367 non-null  float64
 11  noshow_outcome_ratio        38367 non-null  float64
 12  reachability_score          38367 non-null  float64
 13  outcome_trend               383

Adding behavioral data from Booking Calls


In [44]:
behaviour.columns = behaviour.columns.str.lower()
merged_df = final_df.merge(behaviour, 
                          left_on='leadid', 
                          right_on='id', 
                          how='left')

In [45]:
duplicate_requestids = merged_df[merged_df['requestid'].duplicated(keep=False)]['requestid'].unique()
print(f"Duplicate requestid values: {duplicate_requestids}")

# --> loooks ok 

Duplicate requestid values: []


#### Duplicate Check

In [46]:
# Check your actual dataset for duplicates
duplicate_cols = []
cols = merged_df.columns.tolist()

for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        if merged_df.equals(merged_df[cols[j]]):
            duplicate_cols.append((cols[i], cols[j]))

print("Duplicate column pairs:", duplicate_cols)
print(f"Original shape: {merged_df.shape}")

# Remove duplicates if any found
X_final_clean = merged_df.loc[:, ~merged_df.columns.duplicated()]
print(f"After removing duplicates: {X_final_clean.shape}")

Duplicate column pairs: []
Original shape: (14156, 44)
After removing duplicates: (14156, 44)


In [47]:
# Check the original sizes
print("Original DataFrame sizes:")
print(f"final_df: {len(final_df)} rows")
print(f"behaviour: {len(behaviour)} rows")
print(f"merged_df: {len(merged_df)} rows")

print("\n" + "="*50)

# Check how many matches we got
matches = merged_df['id'].notna().sum()
no_matches = merged_df['id'].isna().sum()

print(f"Rows with matches: {matches} ({matches/len(merged_df)*100:.1f}%)")
print(f"Rows without matches: {no_matches} ({no_matches/len(merged_df)*100:.1f}%)")

print("\n" + "="*50)

# Check for duplicates in the key columns before merge
print("Checking for duplicates in join keys:")
print(f"Duplicates in final_df['leadid']: {final_df['leadid'].duplicated().sum()}")
print(f"Duplicates in behaviour['id']: {behaviour['id'].duplicated().sum()}")

print("\n" + "="*50)

# Check the overlap between the two key columns
final_leadids = set(final_df['leadid'])
behaviour_ids = set(behaviour['id'])

print(f"Unique leadids in final_df: {len(final_leadids)}")
print(f"Unique ids in behaviour: {len(behaviour_ids)}")
print(f"Common IDs: {len(final_leadids & behaviour_ids)}")
print(f"IDs only in final_df: {len(final_leadids - behaviour_ids)}")
print(f"IDs only in behaviour: {len(behaviour_ids - final_leadids)}")

Original DataFrame sizes:
final_df: 14156 rows
behaviour: 38367 rows
merged_df: 14156 rows

Rows with matches: 13296 (93.9%)
Rows without matches: 860 (6.1%)

Checking for duplicates in join keys:
Duplicates in final_df['leadid']: 3225
Duplicates in behaviour['id']: 0

Unique leadids in final_df: 10931
Unique ids in behaviour: 38367
Common IDs: 10240
IDs only in final_df: 691
IDs only in behaviour: 28127


In [48]:
# checking if they are converters or not
# Get the rows that didn't match (have NaN in the 'id' column from behaviour)
unmatched_rows = merged_df[merged_df['id'].isna()]

print(f"Analysis of {len(unmatched_rows)} unmatched rows:")
print("="*50)

# Check conversion rate for unmatched entries
conversion_rate_unmatched = unmatched_rows['netcontractsigned'].mean()
total_conversions_unmatched = unmatched_rows['netcontractsigned'].sum()

print(f"Unmatched entries conversion rate: {conversion_rate_unmatched:.3f} ({conversion_rate_unmatched*100:.1f}%)")
print(f"Total conversions among unmatched: {total_conversions_unmatched}")

print("\n" + "="*50)

# Compare with matched entries
matched_rows = merged_df[merged_df['id'].notna()]
conversion_rate_matched = matched_rows['netcontractsigned'].mean()
total_conversions_matched = matched_rows['netcontractsigned'].sum()

print(f"Matched entries conversion rate: {conversion_rate_matched:.3f} ({conversion_rate_matched*100:.1f}%)")
print(f"Total conversions among matched: {total_conversions_matched}")

print("\n" + "="*50)

# Overall comparison
print("COMPARISON:")
print(f"Unmatched: {conversion_rate_unmatched:.3f} ({conversion_rate_unmatched*100:.1f}%)")
print(f"Matched:   {conversion_rate_matched:.3f} ({conversion_rate_matched*100:.1f}%)")
print(f"Difference: {(conversion_rate_matched - conversion_rate_unmatched)*100:.1f} percentage points")

# Statistical significance test (optional)
from scipy import stats
if len(unmatched_rows) > 0 and len(matched_rows) > 0:
    stat, p_value = stats.chi2_contingency([[
        unmatched_rows['netcontractsigned'].sum(), 
        len(unmatched_rows) - unmatched_rows['netcontractsigned'].sum()
    ], [
        matched_rows['netcontractsigned'].sum(), 
        len(matched_rows) - matched_rows['netcontractsigned'].sum()
    ]])[:2]
    
    print(f"\nStatistical test p-value: {p_value:.6f}")
    print(f"Significant difference: {'Yes' if p_value < 0.05 else 'No'}")

Analysis of 860 unmatched rows:
Unmatched entries conversion rate: 0.008 (0.8%)
Total conversions among unmatched: 7.0

Matched entries conversion rate: 0.025 (2.5%)
Total conversions among matched: 337.0

COMPARISON:
Unmatched: 0.008 (0.8%)
Matched:   0.025 (2.5%)
Difference: 1.7 percentage points

Statistical test p-value: 0.002201
Significant difference: Yes


In [49]:
# PROBLEM ONE 
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14156 entries, 0 to 14155
Data columns (total 44 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   requestid                       14156 non-null  int64  
 1   gross_FU                        14156 non-null  int64  
 2   gross_SC                        14156 non-null  int64  
 3   net_FU                          14156 non-null  float64
 4   net_SC                          14156 non-null  float64
 5   time_first_sc_to_first_net_fu   14156 non-null  float64
 6   electricitybill                 14156 non-null  float64
 7   heatingbill                     14156 non-null  float64
 8   netcontractsigned               14156 non-null  float64
 9   selfipa_done                    14156 non-null  int64  
 10  zipregion_missing               14156 non-null  int64  
 11  evaluationtime_missing          14156 non-null  int64  
 12  desiredinstallationend_missing  

In [50]:
# drop non converters with no behavioral data. keep the 7 that converted and recover their data manually
# Get the unmatched rows (the 860 entries without behavior data)
# Get the unmatched rows (the 860 entries without behavior data)
unmatched_rows = merged_df[merged_df['id'].isna()]

print(f"Original unmatched rows: {len(unmatched_rows)}")
print(f"Conversions among unmatched: {unmatched_rows['netcontractsigned'].sum()}")
print(f"Non-conversions among unmatched: {(unmatched_rows['netcontractsigned'] == 0).sum()}")

# Filter unmatched rows to keep only conversions (netcontractsigned == 1)
unmatched_conversions = unmatched_rows[unmatched_rows['netcontractsigned'] == 1]

print(f"\nAfter filtering unmatched rows:")
print(f"Keeping only unmatched conversions: {len(unmatched_conversions)}")

# Get the matched rows (the 13,296 entries with behavior data)
matched_rows = merged_df[merged_df['id'].notna()]

# Combine and replace merged_df
merged_df = pd.concat([matched_rows, unmatched_conversions], ignore_index=True)

print(f"\nFinal merged_df:")
print(f"Matched rows (with behavior): {len(matched_rows)}")
print(f"Unmatched conversions (without behavior): {len(unmatched_conversions)}")
print(f"Total rows: {len(merged_df)}")
print(f"All remaining unmatched rows are conversions: {(merged_df[merged_df['id'].isna()]['netcontractsigned'] == 1).all()}")

Original unmatched rows: 860
Conversions among unmatched: 7.0
Non-conversions among unmatched: 853

After filtering unmatched rows:
Keeping only unmatched conversions: 7

Final merged_df:
Matched rows (with behavior): 13296
Unmatched conversions (without behavior): 7
Total rows: 13303
All remaining unmatched rows are conversions: True


In [51]:
# PROBLEM THREE
# see leads that didnt make it to sc --> le
merged_df.loc[merged_df['net_SC'] == 0, 'requestid']

5        106886
7        106888
20       106902
21       106903
23       106905
          ...  
13284    122728
13285    122729
13289    122733
13290    122734
13291    122735
Name: requestid, Length: 5517, dtype: int64

In [52]:
# any of them have contract signed?  yes but ofc after the cutoff time of the dataset
merged_df.loc[
    (merged_df['net_SC'] == 0) & (merged_df['netcontractsigned'] == 1),
    'requestid'
]

5186     113414
8975     117775
9454     118332
13017    122408
13083    122485
Name: requestid, dtype: int64

In [53]:
# lets drop the ones who havent made to net sc
# Keep rows where net_SC is NOT equal to 0
merged_df = merged_df[merged_df['net_SC'] != 0]

In [54]:
# drop requestid ad other indicatiors
merged_df = merged_df.drop(columns=['requestid', 'id', 'leadid'])

In [55]:
# Check conversions in missing call data rows
missing_mask = merged_df['total_bc_attempts'].isnull()
print(f"Conversions in missing rows: {merged_df[missing_mask]['netcontractsigned'].sum()}")
print(f"Conversion rate in missing rows: {merged_df[missing_mask]['netcontractsigned'].mean():.4f}")
print(f"Conversion rate in complete rows: {merged_df[~missing_mask]['netcontractsigned'].mean():.4f}")

Conversions in missing rows: 7.0
Conversion rate in missing rows: 1.0000
Conversion rate in complete rows: 0.0427


In [56]:
# some of these requests are very old and dont have propre registstration, some have some registration but essentially there are only 7 contract signed insode 860, so essentially dropping as a first step of undersampling.
# ill keep 7 positives and recover their data, drop the rest

In [57]:
# this would drop all entries with any missing values, so lets hold our horses here. 
#merged_df = merged_df.dropna().copy()
print(f"New dataset shape: {merged_df.shape}")
print(f"New conversion rate: {merged_df['netcontractsigned'].mean():.4f}")
print(f"Total conversions kept: {merged_df['netcontractsigned'].sum()}")

New dataset shape: (7786, 41)
New conversion rate: 0.0435
Total conversions kept: 339.0


In [58]:
# Create subfolder if it doesn't exist
os.makedirs('processed_data', exist_ok=True)

# Save your merged dataframe
merged_df.to_csv('processed_data/merged_df.csv', index=False)

print("Dataset saved successfully!")

Dataset saved successfully!
